In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA
import scipy

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)

In [13]:
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/"
protien="FAS_MOUSE"  
time=31
peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((".csv" in x) and ("_1_" in x)) ]
peptides=[x  for x in peptides if 'A' in x]
all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
peptidesinfo=dict()
len(peptides)

13

In [14]:
def charcount(pepSeq):
    res={key: 0 for key in ['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y']}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [15]:
for k in range(len(peptides)):
    data=pd.read_csv(f"C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/{peptides[k]}_{time}_{protien}.csv")
    data["pxt_neh"]=data.NEH*data.pxt
    data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
    minval=min(data.RMSE)
     
    data=data[data.RMSE==minval]
    
    peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                           "charcount":charcount(peptides[k])}


In [16]:
all_AAs

array(['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p',
       'q', 'r', 's', 't', 'v', 'w', 'y'], dtype='<U1')

In [17]:
visited=set()
visited.add(peptides[0].split('_')[1])
coefficients=[]
for k in range(0,len(peptides)-1):
    if(peptides[k+1].split('_')[1] in visited):
        continue
        
    pep1=peptidesinfo[peptides[k]]
    pep2=peptidesinfo[peptides[k+1]]    
    visited.add(peptides[k].split('_')[1])
    
    r=pep1["pxt_neh"]/pep2["pxt_neh"]
    coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
 

In [18]:
coefficients=np.array(coefficients)
coefficients[:,1:].shape

(12, 19)

In [19]:
a_val=4
sol=list(scipy.optimize.nnls(coefficients[:,1:],-a_val*coefficients[:,0])[0])
sol.insert(0,a_val)
sol

[4,
 0.0,
 0.8478517152325133,
 1.786536464002578,
 0.03291893081485633,
 0.612755536252198,
 0.0,
 0.0,
 0.0,
 0.2649816733566618,
 0.0,
 0.0,
 1.5031259409294147,
 3.245441292654486,
 0.6642060915635881,
 1.0954958490599038,
 0.0,
 0.0,
 1.1369275205340381,
 0.0]

In [20]:

t_neh,e_neh=[],[]
for i in range (len(peptides)):
    pep1=peptidesinfo[peptides[i]]
    p_neh=0
    for index,c in enumerate(all_AAs):
#         print(c,pep1["charcount"][c],pep1["charcount"][c]*sol[index])
        p_neh+=(pep1["charcount"][c]*sol[index])
#     print("\n",p_neh,"\t",pep1["T_NEH"])
    t_neh.append(pep1["T_NEH"])
    e_neh.append(p_neh)
    
res=pd.DataFrame([peptides,t_neh,e_neh])
res=res.transpose()
res.columns=["peptides","True_NEH","e_NEH"]
res["dif"]=abs(res.True_NEH-res.e_NEH)
print(np.mean(res.dif),np.median(res.dif),np.std(res.dif))
res.sort_values(by='dif')

14.18331712624166 12.369712011281303 4.123283100058365


,peptides,True_NEH,e_NEH,dif
9,_SDEAVKPLGVK_2,19.0,10.110747,8.889253
10,_SDEAVKPLGVK_3,19.0,10.110747,8.889253
0,_AcVDTALENLSTLK_2,23.0,12.524829,10.475171
11,_VTAIYIDPATHR_3,22.0,11.015184,10.984816
5,_LKEDTQVADVTTSR_3,24.0,12.752365,11.247635
12,_WLSTSIPEAQWQSSLAR_2,38.0,25.630553,12.369447
2,_FPQLDDTSFANSR_2,26.0,13.630288,12.369712
6,_LLLPEDPLISGLLNSQALK_2,33.0,17.5447,15.4553
7,_LLLPEDPLISGLLNSQALK_3,33.0,17.5447,15.4553
1,_DAMLENQTPELFQDVNKPK_3,37.0,19.328793,17.671207


In [21]:
temp=pd.DataFrame(coefficients)
temp.columns=all_AAs
temp 

,a,c,d,e,f,g,h,i,k,l,m,n,p,q,r,s,t,v,w,y
0,1.352941,1.0,-0.294118,-0.294118,-0.647059,0.000000,0.000000,0.000000,-0.294118,1.705882,-0.647059,-0.294118,-1.294118,-1.294118,0.000000,1.000000,1.352941,0.352941,0.000000,0.000000
1,-0.416666,0.0,-0.833332,2.000000,-1.833332,0.000000,0.000000,0.000000,2.000000,0.583334,1.000000,0.583334,0.583334,0.583334,-1.416666,-2.833332,-0.416666,1.000000,0.000000,0.000000
2,-0.632654,0.0,2.000000,-0.816327,1.183673,-2.448981,0.000000,0.000000,0.000000,1.000000,-0.816327,-1.448981,1.000000,0.183673,0.183673,1.183673,-0.632654,0.000000,0.000000,-0.816327
3,1.109091,0.0,-0.890909,-0.781818,0.109091,3.000000,-0.890909,0.000000,0.000000,-0.890909,1.000000,3.000000,-3.563636,1.000000,0.109091,-1.672727,1.109091,-1.781818,0.000000,1.000000
4,-0.464064,0.0,-1.928128,0.535936,1.000000,0.000000,1.000000,0.000000,-1.464064,-0.464064,0.000000,0.000000,4.000000,-1.464064,-0.464064,1.535936,-3.392192,-0.928128,0.000000,0.000000
5,0.272903,0.0,1.272903,0.272903,0.000000,-0.727097,0.000000,-0.727097,0.272903,-4.089677,0.000000,-0.727097,-1.454194,0.272903,1.000000,-0.454194,3.000000,2.000000,0.000000,0.000000
6,-0.020408,0.0,-0.020408,-0.020408,0.000000,-0.020408,0.000000,-0.020408,-0.020408,-0.142857,0.000000,-0.020408,-0.040816,-0.020408,0.000000,-0.040816,0.000000,0.000000,0.000000,0.000000
7,0.079394,0.0,1.000000,0.079394,0.000000,-0.841212,0.000000,0.079394,1.000000,5.158788,0.000000,0.079394,2.000000,-0.841212,-0.920606,-1.682424,-0.920606,-2.761818,0.000000,0.000000
8,-0.887872,0.0,-1.887872,-0.887872,0.000000,0.112128,0.000000,1.000000,-3.775743,0.112128,0.000000,1.000000,-1.887872,2.000000,1.000000,2.112128,1.000000,-0.775743,0.000000,0.000000
9,-0.028235,0.0,-0.028235,-0.028235,0.000000,-0.028235,0.000000,0.000000,-0.056471,-0.028235,0.000000,0.000000,-0.028235,0.000000,0.000000,-0.028235,0.000000,-0.056471,0.000000,0.000000


In [22]:
for c in range( len(coefficients)):
    print(np.dot(coefficients[c],sol))

0.017962922786876057
0.019230038226134116
0.027940691892183733
0.018881625317154427
0.011745649454280027
-0.004330036191563025
-0.3580553274138128
-0.01961888371499798
-0.008708907278617062
-0.2854816379820809
-0.02226209370242005
3.3295233259524646e-16
